# Урок 2. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [20]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [27]:
wine = load_wine()

wine_simple = np.delete(wine.data, [0,1,2,3,4,5,6,7,8,10],axis=1)

x_train, x_test, y_train, y_test = train_test_split(wine_simple, wine['target'], random_state=17, train_size=0.8)
x_train.shape

(142, 3)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [28]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [31]:
skf = StratifiedKFold(n_splits=5, random_state=17)

E:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [39]:
lgst = LogisticRegression(random_state=17, multi_class='ovr')
model = lgst.fit(x_train, y_train)
print(cross_val_score(model, x_train, y_train, cv=skf))
np.mean(cross_val_score(model, x_train, y_train, cv=skf))




[0.93103448 0.82758621 0.96428571 0.92857143 0.89285714]


0.9088669950738917

### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [41]:
from sklearn.svm import SVC

In [46]:
svc = SVC(random_state=17)
model = svc.fit(x_train, y_train)
print(cross_val_score(model, x_train, y_train, cv=skf))
print('train data',np.mean(cross_val_score(model, x_train, y_train, cv=skf)))

print('test data',np.mean(cross_val_score(model, x_test, y_test, cv=skf)))



[0.75862069 0.51724138 0.71428571 0.64285714 0.71428571]
train data 0.6694581280788178
test data 0.6928571428571428


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [47]:
from sklearn.metrics import classification_report, confusion_matrix

In [51]:
lgst_pred = lgst.predict(x_test)
svc_pred = svc.predict(x_test)

print(confusion_matrix(y_test, lgst_pred))
print('\n\n', confusion_matrix(y_test, svc_pred))

#Метод опорных веткоров сильно напутал в 3 классе. Ни одного объекта истинно 3 класса не отнес к 3 классу

[[ 8  0  0]
 [ 0 16  0]
 [ 0  0 12]]


 [[ 8  0  0]
 [ 0 16  0]
 [ 1 11  0]]


### 5.
Для каждой модели выведите classification report.

In [54]:
print(classification_report(y_test, lgst_pred))
print(classification_report(y_test, svc_pred))






              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        12

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.59      1.00      0.74        16
           2       0.00      0.00      0.00        12

    accuracy                           0.67        36
   macro avg       0.49      0.67      0.56        36
weighted avg       0.46      0.67      0.54        36



E:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
